In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ARCUS, ADBenchDataset
from streamvigil.detectors import BasicAutoEncoder, BasicDetector

In [2]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
    config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [3]:
# Anomaly Detector
auto_encoder = BasicAutoEncoder(
    encoder_dims=[500, 450, 400, 350, 300, 250],
    decoder_dims=[250, 300, 350, 400, 450, 500],
    batch_norm=True,
)
detector = BasicDetector(auto_encoder)

arcus = ARCUS(detector, max_epochs=20)

In [4]:
random_state = 42

# Load dataset
data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    test_size=0.01,
    random_state=random_state,
)

# DataLoader
loader = DataLoader(data, batch_size=1024)

In [5]:
# Initialize model pool
logger.info("Start initializing model pool...")
X, _ = next(iter(loader))
arcus.init(X)
logger.info("Completed initializing model pool!")

2024-04-24 22:22:38,519 [INFO] __main__: Start initializing model pool...
2024-04-24 22:22:38,883 [INFO] streamvigil._arcus: loss: 0.236929
2024-04-24 22:22:39,182 [INFO] streamvigil._arcus: loss: 0.114899
2024-04-24 22:22:39,475 [INFO] __main__: Completed initializing model pool!


In [6]:
# ARCUS simulation
metrics = BinaryAUROC()

logger.info("Start ARCUS simulation...")
for X, y in loader:
    scores = arcus.run(X)
    metrics.update(scores, y)

# Compute AUROC score
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed ARCUS simulation!")

2024-04-24 22:22:55,279 [INFO] __main__: Start ARCUS simulation...
2024-04-24 22:22:55,297 [INFO] streamvigil._arcus: Start training model with id 7b757927-a735-402f-aeae-6245b0ed7443...
2024-04-24 22:22:55,335 [INFO] streamvigil._arcus: loss: 0.064705
2024-04-24 22:22:55,602 [INFO] streamvigil._arcus: loss: 0.041510
2024-04-24 22:22:55,870 [INFO] streamvigil._arcus: Completed training model with id 7b757927-a735-402f-aeae-6245b0ed7443!
2024-04-24 22:22:55,882 [INFO] streamvigil._arcus: Concept drift detected!
2024-04-24 22:22:55,886 [INFO] streamvigil._arcus: Start training a new model...
2024-04-24 22:22:55,922 [INFO] streamvigil._arcus: loss: 0.236608
2024-04-24 22:22:56,225 [INFO] streamvigil._arcus: loss: 0.114418
2024-04-24 22:22:56,458 [INFO] streamvigil._arcus: Completed training new model!
2024-04-24 22:22:56,519 [INFO] streamvigil._arcus: Start training model with id b6851ce5-8760-425c-b4d4-d8fce8a82762...
2024-04-24 22:22:56,546 [INFO] streamvigil._arcus: loss: 0.061248
2024